# Data Extraction, Transformation and Loading

## Objectives

* Data cleaning and transformation to prepare data for analysis

## Inputs

* Raw data loaded from Health Insurance csv from Kaggle (local copy saved in data folder)

## Outputs

* Cleaned data is saved as insurance-cleaned.csv in the data folder
* Dashboard ready transformed data is saved to dashboard_data.csv in the data folder

# Import data and packages

Install any packages needed:

In [ ]:
# Import packages used in the notebook

import pandas as pd
import numpy as np

Import raw data

In [ ]:
# import data frame from csv file and display first 5 rows
df = pd.read_csv('../data/insurance.csv')
df.head()

## Data Transformation

### Transform part one: Clean Data: Remove duplicates and handle missing values.

Firstly, we checked for missing data. It turned out that there were no (0) missing values for the data analysed for any of the categories.

In [ ]:
#Find missing data (code from the LMS and ChatGPT helped me make this):
df.isnull().sum()

We then searched for duplicate data and removed it (ChatGPT helped me with this)

In [ ]:
# 🔹 Find and print duplicate rows (excluding first occurrence)
duplicates = df[df.duplicated()]
print("\nDuplicate rows:")
print(duplicates)

# 🔹 Remove duplicates (keep first occurrence)
df_no_duplicates = df.drop_duplicates()

print("\nDataFrame after removing duplicates:")
df_no_duplicates.head()


### Transform data part two: Categorise data and check for outliers

We've added in a category for BMI (ChatGPT suggested some good category values for us and helped write quick code for it). Unreasonable values for when the data was collected (e.g someone having an age of 130) were also checked for and none appeared.

In [ ]:
# Define a function to categorize BMI
def categorize_bmi(bmi):
    if bmi < 18.5:
        return "Underweight"
    elif 18.5 <= bmi < 25:
        return "Normal"
    elif 25 <= bmi < 30:
        return "Overweight"
    else:
        return "Obese"

# Apply function to create a new column
df['bmi_category'] = df['bmi'].apply(categorize_bmi)

#Clean dataframe in place: fix types, strip spaces, handle missing, encode. (code from  ChatGPT helped me make this):
def transform_clean(df):
    
    df = df.copy()
    # strip and lower string columns
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].astype(str).str.strip()
    # numeric coercion for known numeric columns
    for col in ['age','bmi','children','charges']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    # mapping / encoding (safe)
    if 'smoker' in df.columns:
        df['smoker'] = df['smoker'].str.lower().map({'no':0,'yes':1})
    return df    

#Check for unreasonable values (e.g unnrealistic values for BMI and age at the time of dataset being collected) and handle them.
def validate_df(df):
    errors = []
    # shape
    if df.empty:
        errors.append("DataFrame is empty.")
    # ranges
    if df['age'].min() < 0 or df['age'].max() > 130:
        errors.append("Age out of expected range.")
        print(errors)
    if (df['bmi'] < 10).any() or (df['bmi'] > 80).any():
        errors.append("BMI has out-of-range values.")
        print(errors)
    if (df['children'] < 0).any() or (df['children'] > 20).any():
        errors.append("Children out-of-range.")
        print(errors)
# === ETL Pipeline Wrapper ===
def etl_pipeline(filepath: str):
    """
    End-to-end ETL pipeline:
    - Extract: Load dataset from CSV
    - Transform: Clean, encode, and add BMI categories
    - Load: Return a ready-to-analyze DataFrame
    - Validate: Run quality checks and print issues
    """
    # Extract
    df = pd.read_csv(filepath)

    # Transform (clean + encode)
    df = transform_clean(df)

    # Add BMI categories
    df["bmi_category"] = df["bmi"].apply(categorize_bmi)

    # Validate
    validate_df(df)

    # Load (return cleaned DataFrame)
    return df

# === Run the ETL pipeline ===
df_clean = etl_pipeline("../data/insurance.csv")

# Quick check
df_clean.head()

# Show first 10 rows to check
transform_clean(df)
validate_df(df)
print(df.head(10))

Code has also been used to automate the ETL process and makes it reusable for future datasets.

In [ ]:
# -----------------------------
# ADDITION: Automated ETL Pipeline 
# -----------------------------
def etl_pipeline(file_path):
    """
    Automated ETL pipeline for insurance dataset:
    - Extract
    - Transform
    - Load
    - Validate
    """
    df = pd.read_csv(file_path)
    
    # Remove duplicates
    df = df.drop_duplicates()
    
    # BMI categories
    def categorize_bmi(bmi):
        if bmi < 18.5: return "Underweight"
        elif 18.5 <= bmi < 25: return "Normal"
        elif 25 <= bmi < 30: return "Overweight"
        else: return "Obese"
    df['bmi_category'] = df['bmi'].apply(categorize_bmi)
    
    # Encode categorical variables
    df['smoker'] = df['smoker'].str.lower().map({'no':0,'yes':1})
    df['region'] = df['region'].astype('category').cat.codes
    
    # Validate ranges
    assert df['age'].between(0,130).all(), "Age out of range"
    assert df['bmi'].between(10,80).all(), "BMI out of range"
    assert df['children'].between(0,20).all(), "Children out of range"
    
    return df

# Usage
df_clean = etl_pipeline('../data/insurance.csv')
df_clean.head()

## Export Cleaned Data

In [ ]:
# export cleaned data to csv file in data folder
df_clean.to_csv("../data/insurance-cleaned.csv",  index=False)

# 🖥️ Dashboard Preparation

This code block prepares the dataset for a business intelligence dashboard (e.g., Tableau or Power BI).  

**Steps included:**

1. Identify continuous and categorical columns based on the dataset guide.  
2. Scale continuous numeric columns (`age` and `bmi`) for better visual consistency.  
3. Ensure categorical columns, including `bmi_category`, are treated as strings for BI tools.  
4. Precompute aggregations (mean `charges`) by `region`, `smoker`, `children`, and `bmi_category`.  
5. Export the cleaned, dashboard-ready dataset to `dashboard_data.csv`.



In [ ]:
# -------------------------------
# 1️⃣ Define columns based on data guide
# -------------------------------
continuous_cols = ['age', 'bmi']
categorical_cols = ['sex', 'children', 'smoker', 'region']
target_col = 'charges'

print("Continuous numeric columns to scale:", continuous_cols)
print("Categorical columns for BI:", categorical_cols + ['bmi_category'])

# -------------------------------
# 2️⃣ Scale continuous numeric columns
# -------------------------------
df_scaled = df_clean.copy()  # Keep original data intact

for col in continuous_cols:
    mean = df_scaled[col].mean()
    std = df_scaled[col].std()
    df_scaled[col] = (df_scaled[col] - mean) / std

print("\nScaled continuous numeric features:\n")
print(df_scaled[continuous_cols].head())

# -------------------------------
# 3️⃣ Ensure categorical columns are strings for BI
# -------------------------------
for col in categorical_cols + ['bmi_category']:  # include existing BMI category
    df_scaled[col] = df_scaled[col].astype(str)

# -------------------------------
# 4️⃣ Precompute aggregations for dashboard visualisations
# -------------------------------
aggregations = {
    'region': df_scaled.groupby('region')[target_col].mean().reset_index(),
    'smoker': df_scaled.groupby('smoker')[target_col].mean().reset_index(),
    'children': df_scaled.groupby('children')[target_col].mean().reset_index(),
    'bmi_category': df_scaled.groupby('bmi_category')[target_col].mean().reset_index()
}

print("\nAggregations ready for dashboard visualisations.")

# -------------------------------
# 5️⃣ Optionally export for Tableau / Power BI
# -------------------------------
df_scaled.to_csv('../data/dashboard_data.csv', index=False)
print("\nDashboard-ready dataset saved as 'dashboard_data.csv'.")


# Conclusions and Next Steps
* ETL Pipeline created for future use.
* Data checked for outliers, duplicates and unreasonable values. 
* Feature engineering used to categorize BMI.
* Data Cleaned and ready for use in visualisations / dashboarding.
* Data saved in relevant folder, seperated with original file, cleaned file and dashboard file.










